In [1]:
import numpy as np

import random
import h5py
from keras.models import *
from keras.layers import *
from keras.layers.core import *
from keras.layers.normalization import *
from keras.optimizers import *
from keras.callbacks import *
from keras import backend as K
from keras.regularizers import *
import theano.tensor as T
import theano

import os
import random
import time
import matplotlib
import matplotlib.pyplot as plt

import scipy.signal as sig
import operator
import math
import re

# for reproducibility
np.random.seed(1337) 
random.seed(1337)

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX 690 (CNMeM is disabled, cuDNN 5105)


In [2]:
# external custom code I wrote
from load_data import *
from windowing import *
from consts import *
from utility import *
from pesq import *
from nn_blocks import *

In [3]:
train_X = np.load('./pesq_dataset/train_X.npy')
train_y = np.load('./pesq_dataset/train_y.npy')
val_X = np.load('./pesq_dataset/val_X.npy')
val_y = np.load('./pesq_dataset/val_y.npy')

In [4]:
train_X = np.reshape(train_X, (2, -1, WINDOW_SIZE, 1))
val_X = np.reshape(val_X, (2, -1, WINDOW_SIZE, 1))
train_X = [train_X[0], train_X[1]]
val_X = [val_X[0], val_X[1]]

In [5]:
NFILTS = 32
FSIZE = 5
DENSE_SIZE = 32

def siamese_half():
    inp = Input(shape = (WINDOW_SIZE, 1))
    ret = Reshape((WINDOW_SIZE, 1))(inp)
    
    ret = channel_increase_block(NFILTS, FSIZE)(ret)
    
    ret = residual_block(NFILTS, FSIZE, dilation = 1)(ret)
    ret = residual_block(NFILTS, FSIZE, dilation = 1)(ret)
    
    ret = channel_decrease_block(NFILTS, FSIZE)(ret)
    
    ret = Flatten()(ret)
    ret = Dense(DENSE_SIZE, activation = 'linear', kernel_initializer = W_INIT)(ret)
    
    return Model(inputs = inp, outputs = ret)


input_orig = Input(shape = (WINDOW_SIZE, 1))
input_dirty = Input(shape = (WINDOW_SIZE, 1))

base_network = siamese_half()
processed_a = base_network(input_orig)
processed_b = base_network(input_dirty)

out = EuclideanDistance()([processed_a, processed_b])
out = Dense(1, activation = 'sigmoid', kernel_initializer = W_INIT)(out)

model = Model(inputs = [input_orig, input_dirty], outputs = out)

model.compile(loss = ['mae'], optimizer = Adam(), metrics = ['mse'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 512, 1)        0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 512, 1)        0                                            
____________________________________________________________________________________________________
model_1 (Model)                  (None, 32)            47681                                        
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 1)             0                                            
___________________________________________________________________________________________

In [6]:
model.fit(train_X, train_y,
          batch_size = 128,
          epochs = 10,
          verbose = 1,
          shuffle = True,
          validation_data = (val_X, val_y))

Train on 129597 samples, validate on 135616 samples
Epoch 1/10
129597/129597 [==============================] - 521s - loss: 0.1928 - mean_squared_error: 0.0619 - val_loss: 0.1671 - val_mean_squared_error: 0.0405
Epoch 2/10
129597/129597 [==============================] - 524s - loss: 0.1281 - mean_squared_error: 0.0285 - val_loss: 0.1393 - val_mean_squared_error: 0.0284
Epoch 3/10
129597/129597 [==============================] - 524s - loss: 0.1038 - mean_squared_error: 0.0213 - val_loss: 0.1334 - val_mean_squared_error: 0.0287
Epoch 4/10
129597/129597 [==============================] - 523s - loss: 0.0954 - mean_squared_error: 0.0205 - val_loss: 0.1338 - val_mean_squared_error: 0.0305
Epoch 5/10
129597/129597 [==============================] - 526s - loss: 0.0920 - mean_squared_error: 0.0208 - val_loss: 0.1356 - val_mean_squared_error: 0.0318
Epoch 6/10
129597/129597 [==============================] - 524s - loss: 0.0902 - mean_squared_error: 0.0210 - val_loss: 0.1373 - val_mean_squa

KeyboardInterrupt: 

In [7]:
model.save('pesq_model.h5')

In [8]:
# number of speech files for train, val, and test
TRAIN_SIZE = 100
VAL_SIZE = 100
TEST_SIZE = 100

[train_paths, val_paths, test_paths], \
[train_waveforms, val_waveforms, test_waveforms], \
[train_procwave, val_procwave, test_procwave], \
[train_wparams, val_wparams, test_wparams], \
[train_windows, val_windows, test_windows] = load_data(TRAIN_SIZE, VAL_SIZE, TEST_SIZE)

In [9]:
orig_windows = np.copy(train_windows[0])
corrupted_windows = np.copy(orig_windows)

crange = WINDOW_SIZE / 2
corrupted_windows *= np.random.normal(1.0, 0.1, corrupted_windows.shape)

In [10]:
print run_pesq_windows(orig_windows, corrupted_windows, train_wparams[0], train_wparams[0])
print mse(orig_windows, corrupted_windows)
print avgErr(orig_windows, corrupted_windows)

2.157
0.000107495
0.00479659


In [11]:
orig_windows = np.reshape(orig_windows, (-1, WINDOW_SIZE, 1))
corrupted_windows = np.reshape(corrupted_windows, (-1, WINDOW_SIZE, 1))

pred = np.mean(model.predict([orig_windows, corrupted_windows]))
pred = pred * 3.5 + 1.0

print pred

2.85794138908
